In [2]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

In [5]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

x = torch.rand(2, 20)
net(x)

tensor([[ 0.1056, -0.1350,  0.0974,  0.0291, -0.2043, -0.2150, -0.0959, -0.2659,
          0.1975,  0.0070],
        [ 0.1318, -0.1368,  0.0502,  0.0145, -0.1564, -0.0877,  0.0187, -0.2783,
          0.2295,  0.2346]], grad_fn=<AddmmBackward0>)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(x)

tensor([[ 0.0190,  0.1411, -0.1954, -0.3578,  0.0451, -0.0216, -0.1178,  0.0526,
         -0.0683, -0.0439],
        [ 0.0311,  0.0239, -0.1195, -0.2888,  0.0495,  0.0273,  0.0865,  0.0535,
         -0.0441, -0.0593]], grad_fn=<AddmmBackward0>)

In [17]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [15]:
X = torch.rand(2, 20)

In [18]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2351,  0.1216,  0.0800,  0.1367, -0.0520, -0.0776, -0.1547, -0.1433,
          0.2471, -0.0725],
        [-0.2359,  0.0422, -0.0008,  0.0718, -0.0391, -0.1894, -0.0656, -0.0281,
          0.1932, -0.1179]], grad_fn=<AddmmBackward0>)

In [22]:
class FixedHiddenNLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X/=2
        return X.sum()
        

In [38]:
net = FixedHiddenNLP()
net(x)

tensor(0.2282, grad_fn=<SumBackward0>)

In [39]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [42]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenNLP())
chimera(x)

tensor(0.0872, grad_fn=<SumBackward0>)

In [ ]:
import torch
from torch import nn

In [45]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size = (2, 4))
net(X)

tensor([[0.0110],
        [0.0253]], grad_fn=<AddmmBackward0>)

In [46]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2176,  0.0551,  0.2224,  0.1384,  0.0357, -0.2241, -0.0518,  0.3219]])), ('bias', tensor([0.1141]))])


In [47]:
print(net[2].bias)

Parameter containing:
tensor([0.1141], requires_grad=True)


In [49]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

In [52]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2595],
        [-0.2595]], grad_fn=<AddmmBackward0>)

In [53]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [54]:
rgnet[0][1][0].bias.data

tensor([ 0.4475,  0.4209, -0.1557,  0.0290, -0.1452,  0.4734,  0.3324,  0.3887])

In [55]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)

In [56]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0094, 0.0107, 0.0162, 0.0054]), tensor(0.))

In [64]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [65]:

net.apply(init_constant)
net[0].weight.data[0]

tensor([1., 1., 1., 1.])

In [66]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)